# Huggingface + PhoBERT + Captum

In [1]:
import pandas as pd
from underthesea import word_tokenize, sent_tokenize, text_normalize
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from captum.attr import  LayerIntegratedGradients, visualization as viz

## 1. Input Data

In [2]:
df = pd.read_csv('data/df_sentiment.csv')
print(df.shape)
df.head()

(734, 2)


,text,labels
0,#khiếu_nại Một ngày tự nhiên thấy tài khoản bị...,negative
1,"Shop mình lập đc 4 năm rùi,bán trên này phục v...",negative
2,#hopdap #bucxucMình có bán hàng order qua shop...,negative
3,"em mới lập nick shopee này được vài tháng, bán...",negative
4,Shop mình tự nhiên bị shopee khóa vì gian lận ...,negative


In [3]:
def apply_word_tokenize(sen):
    sen = " ".join(sen.split())
    sens = sent_tokenize(sen)
    tokenized_sen = []
    for sen in sens:
        tokenized_sen += word_tokenize(text_normalize(sen))
    return ' '.join(['_'.join(words.split(' ')) for words in tokenized_sen])

In [4]:
df['token'] = df['text'].map(lambda x: apply_word_tokenize(x.lower()))
df.head()

,text,labels,token
0,#khiếu_nại Một ngày tự nhiên thấy tài khoản bị...,negative,# khiếu_nại một ngày tự_nhiên thấy tài_khoản b...
1,"Shop mình lập đc 4 năm rùi,bán trên này phục v...",negative,"shop mình lập đc 4 năm rùi , bán trên này phục..."
2,#hopdap #bucxucMình có bán hàng order qua shop...,negative,# hopdap # bucxucmình có bán hàng order qua sh...
3,"em mới lập nick shopee này được vài tháng, bán...",negative,"em mới lập nick shopee này được vài tháng , bá..."
4,Shop mình tự nhiên bị shopee khóa vì gian lận ...,negative,shop mình tự_nhiên bị shopee khóa vì gian_lận ...


In [5]:
label_encoder = LabelEncoder()
df_preprocess = df[['token', 'labels']].copy()
df_preprocess['labels'] = label_encoder.fit_transform(df['labels'])
df_preprocess.rename(columns={'token': 'text'}, inplace=True)
df_preprocess.head()

,text,labels
0,# khiếu_nại một ngày tự_nhiên thấy tài_khoản b...,0
1,"shop mình lập đc 4 năm rùi , bán trên này phục...",0
2,# hopdap # bucxucmình có bán hàng order qua sh...,0
3,"em mới lập nick shopee này được vài tháng , bá...",0
4,shop mình tự_nhiên bị shopee khóa vì gian_lận ...,0


In [6]:
labels = df['labels'].unique().tolist()
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}

id2label

{0: 'negative', 1: 'neutral', 2: 'positive'}

## 2 Models

In [7]:
pretrain_name = "phobert/sentiment"
tokenizer = AutoTokenizer.from_pretrained(pretrain_name)
model = AutoModelForSequenceClassification.from_pretrained(
    pretrain_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model.to(device)
model.eval()
model.zero_grad()

cuda:0


In [9]:
class XAI:
    def __init__(self, text_, label_, tokenizer_, model_, device_):
        self.text = text_
        self.label = label_
        self.tokenizer = tokenizer_
        self.model = model_
        self.ref_token_id = self.tokenizer.pad_token_id
        self.sep_token_id = self.tokenizer.sep_token_id
        self.cls_token_id = self.tokenizer.cls_token_id
        self.device = device_
        self.input_ids = None
        self.ref_input_ids = None
        self.sep_id = None

    def construct_input_ref_pair(self):
        text_ids = self.tokenizer.encode(self.text, add_special_tokens=False)
        input_ids = [self.cls_token_id] + text_ids + [self.sep_token_id]
        ref_input_ids = [self.cls_token_id] + [self.ref_token_id] * len(text_ids) + [self.sep_token_id]

        self.input_ids = torch.tensor([input_ids], device=device)
        self.ref_input_ids = torch.tensor([ref_input_ids], device=device)
        self.sep_id = len(text_ids)

        return self.input_ids, self.ref_input_ids, self.sep_id

    def custom_forward(self, inputs):
        preds = self.model(inputs)[0]
        return torch.softmax(preds, dim=1)[0][1].unsqueeze(-1)

    def process(self):
        indices = self.input_ids[0].detach().tolist()
        all_tokens = tokenizer.convert_ids_to_tokens(indices)

        lig = LayerIntegratedGradients(self.custom_forward, self.model.roberta.embeddings)
        attributions, delta = lig.attribute(inputs=self.input_ids,
                                            baselines=self.ref_input_ids,
                                            n_steps=150,
                                            internal_batch_size=3,
                                            return_convergence_delta=True)

        attributions = attributions.sum(dim=-1).squeeze()
        attributions_sum = attributions / torch.norm(attributions)

        score_bert = torch.softmax(model(self.input_ids)[0], dim=1).cpu()
        prod_pred = score_bert[0][self.label]
        class_pred = score_bert.argmax()

        print(f'Text: {text} \n'
              f'Predicted Probability: {prod_pred}\n'
              f'Predicted Class: {class_pred} ({id2label[class_pred.item()]}) vs. True Class: {self.label} ({id2label[self.label]})')

        score_vis = viz.VisualizationDataRecord(attributions_sum,
                                                pred_prob=prod_pred,
                                                pred_class=class_pred,
                                                true_class=self.label,
                                                attr_class=self.text,
                                                attr_score=attributions_sum.sum(),
                                                raw_input_ids=all_tokens,
                                                convergence_score=delta)

        viz.visualize_text([score_vis])

In [17]:
for i in [0, 233, 10]:
    text = df_preprocess['text'].values[i]
    label = df_preprocess['labels'].values[i]
    explain = XAI(text, label, tokenizer, model, device)
    explain.construct_input_ref_pair()
    explain.process()
    print(10*'=')

Text: # khiếu_nại một ngày tự_nhiên thấy tài_khoản bị khóa , gọi lên tổng_đài hỏi thì nhận được câu trả_lời là do tài_khoản có dấu_hiệu hack số like với follow ở shopee và không hỗ_trợ mở lại 😂 
Predicted Probability: 0.990982711315155
Predicted Class: 0 (negative) vs. True Class: 0 (negative)


Text: ngày bán được nhiều nhất sau 3 năm bán shopee nên khoe tý bắt_đầu quảng_cáo ngày 200 k 
Predicted Probability: 0.9635687470436096
Predicted Class: 2 (positive) vs. True Class: 2 (positive)


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (0.96),ngày bán được nhiều nhất sau 3 năm bán shopee nên khoe tý bắt_đầu quảng_cáo ngày 200 k,-3.57,#s ngày bán được nhiều nhất sau 3 năm bán sho@@ pee nên khoe tý bắt_đầu quảng_cáo ngày 200 k #/s


Text: tận_cùng của bất_lực 😣_😣_😣 
Predicted Probability: 0.9867959022521973
Predicted Class: 0 (negative) vs. True Class: 0 (negative)


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.99),tận_cùng của bất_lực 😣_😣_😣,-0.90,#s tận_cùng của bất_lực #unk _@@ #unk _@@ #unk #/s


In [18]:
df_preprocess.query('labels == 1')

,text,labels
224,"dạ chào anh_chị , anh_chị cho em hỏi chút shop...",1
225,các bác cho em hỏi là nếu cuối tháng em đủ chỉ...,1
226,các anh_chị cho em hỏi về vấn_đề dưới đây ạ .,1
227,dạ cho em hỏi có anh_chị nào bị vấn_đề này khô...,1
229,mình bị trừ phí gì tận 123 k thế ạ,1
...,...,...
729,"hôm_qua e ko có nổi 1 đơn , cứ lo là shop bị l...",1
730,hôm_nay mang hàng xuống cho shipper mà buồn gh...,1
731,🎄 make your yard a winter wonderland 🎄 this is...,1
732,cần công_đồng chung tay hỗ_trợ cùng nhau shop ...,1
